In [26]:
library(tidyverse)

# Pairwise urban-rural pi and Fst within cities

In this notebook, we'll examine urban-rural differences in diversity (theta_pi) and urban-rural Fst across all 26 cities. 

## Description of the data

- Low coverage individuals have been removed from these analyses, so we're using the samples that are part of the *finalSamples_lowCovRemoved* sample set from the previous analyses. 
- All analyses were performed using genome-wide 4-fold degenerate site.

The basic workflow is as follows:

1. Generate the Site Allele Frequency (SAF) likelihood distribution for each habitat within each city. 

    - Used the same filters as previously, the most important of which is that sites are only included if 50% of individuals have data. Remember, this is 50% of individuals _within_ a habitat; this would correspond to 5 individuals if none have been removed due to low coverage. 

2. To estimate diversity, generate the folded, one-dimensional SFS from the SAF file in step 1 and estimate diversity separately in urban and rural habitats
3. To estimate Fst, generate the folded, two-dimensional joint SFS of urban-rural habitats and estimate Fst. This uses only sites that are shared between both populations (i.e., the intersection of the two SAF files). 

    - For comparison, I estimated Fst using both Weir and Cockerham (1984) and Hudson (1992).

## Pairwise diversity

### Load diversity data

In [146]:
# Function to load diversity estimates by city and habitat
load_pairwise_diversity <- function(path){
    
    # Get city and site names from file
    city <- dirname(path)
    site <- str_extract(basename(path), pattern = '(?<=_)[r|u]')
    
    full_path <- paste0(inpath, path)
    df <- suppressMessages(read_delim(full_path, delim = '\t')) %>% 
        mutate(tp_scaled = tP / nSites,
               tw_scaled = tW / nSites,
               city = city,
               habitat = site) %>% 
    dplyr::select(city, habitat, tp_scaled, tw_scaled, nSites) %>% 
    group_by(city, habitat) %>% 
    
    # Mean across chromosomes
    summarise(tp_scaled = mean(tp_scaled),
              tw_scaled = mean(tw_scaled),
              total_sites = sum(nSites),
              .groups = 'drop')
    return(df)
    
}

In [147]:
# Merge diversity and habitat diversity dfs into single dataframe
inpath <- '../results/angsd/summary_stats/thetas/by_city/'
div_df <- list.files(inpath, pattern = '*.pestPG', recursive = TRUE) %>% 
    map_dfr(., load_pairwise_diversity) %>% 

    # Calculate urban-rural difference in theta
    pivot_wider(names_from = habitat, values_from = c(tp_scaled, tw_scaled, total_sites)) %>% 
    mutate(tp_diff = tp_scaled_u - tp_scaled_r)
    

In [148]:
head(div_df)

### Plot differences in diversity

In [151]:
# Histogram of urban-rural differences in diversity
div_diff_hist <- div_df %>% 
    ggplot(., aes(x = tp_diff)) +
        geom_histogram(color = 'black', fill = 'white', bins = 13) +
        ylab("Number of cities") + xlab("Urban - rural difference in diversity") +
        theme_classic()
div_diff_hist

### Does difference in neutral diversity predict HCN clines?

In [152]:
slopes <- suppressMessages(read_csv('../../sequencing-prep/data/raw/linearClinesSummary.csv')) %>% 
    dplyr::select(city, betaHCN)
div_df_withSlopes <- div_df %>% 
    left_join(., slopes, by = 'city')

In [153]:
div_mod <- lm(betaHCN ~ tp_diff, data = div_df_withSlopes)
summary(div_mod)

In [154]:
# Looks fine
par(mfrow = (c(2,2)))
plot(div_mod)

In [155]:
# Looks fine
qplot(as.numeric(residuals(div_mod))) + 
    geom_histogram(color = 'black', fill = 'white') + 
    theme_classic()

## Fst

### Load Fst dataframe

In [56]:
# Function to load Fst df by city/habitat
load_fst <- function(path){
    
    # Get Fst type and city from filenames
    dir <- str_split(path, pattern = '/')
    fst_type <- dir[[1]][[1]]
    city <- dir[[1]][[2]]    
    
    full_path <- paste0(inpath, path)
    colnames <- c('chrom', 'pos', 'num', 'denom')
    df <- suppressMessages(read_delim(full_path, delim = '\t', col_names = colnames)) %>% 
        
        # Cap numerators at 0 if negative 
        # https://github.com/ANGSD/angsd/issues/309
        # Does not affect overall pattern
#         mutate(num = ifelse(num < 0, 0, num)) %>% 
        
        # Estimate weighted Fst as ratio of averages
        # https://github.com/ANGSD/angsd/issues/61
        summarise(num_sum = sum(num),
                  denom_sum = sum(denom),
                  fst = num_sum / denom_sum,
                  nSites = n()) %>% 
        mutate(fst_type = ifelse(fst_type == 'fst0', 'wc', 'hudson'),
               city = city)
    
    return(df)
    
}

In [77]:
# Function to get number of samples buy city/habitat
get_num_samples <- function(path){
    
    city <- dirname(path)
    habitat <- str_extract(basename(path), pattern = '(?<=_)[r|u]')  
    
    full_path <- paste0(inpath, path)
    df <- suppressMessages(read_table(full_path, col_names = FALSE)) 
    nsamples <- nrow(df)
    
    df <- data.frame(city = city, habitat = habitat, n = nsamples)
    return(df)
    
}

In [81]:
# Merged df with sample size info
inpath <- '../results/program_resources/bam_lists/by_city/'
sample_size_df <- list.files(inpath, pattern = '*.list', recursive = TRUE) %>% 
    map_dfr(., get_num_samples) %>% 
    pivot_wider(names_from = habitat, values_from = n)
head(sample_size_df)

In [114]:
# Merge Fst dataframes
inpath <- '../results/angsd/summary_stats/fst/'
fst_df <- list.files(inpath, pattern = '*_readable.fst', recursive = TRUE) %>% 
    map_dfr(., load_fst)

In [122]:
fst_df_withSampleSize <- fst_df %>% 
    left_join(., sample_size_df, by = 'city') %>% 
    mutate(nmax = pmax(r, u),
           nmin = pmin(r, u),
           ndiff = nmax - nmin) %>% 
    rowwise() %>% 
    mutate(nmean = mean(c(r, u)))

In [123]:
head(fst_df_withSampleSize)

### WC vs. Hudson's Fst

In [125]:
wc <- fst_df_withSampleSize %>% filter(fst_type == 'wc') %>% pull(fst)
hudson <- fst_df_withSampleSize %>% filter(fst_type == 'hudson') %>% pull(fst)

In [126]:
wc_vs_hudson <- qplot() + 
    geom_point(aes(x = wc, y = hudson), size = 2, alpha = 0.5) +
    geom_abline(slope = 1, intercept = 0) +
    xlab('Weir and Cockerham Fst') + ylab('Hudson Fst') +
    theme_classic()
wc_vs_hudson

In [89]:
outpath <- snakemake@output[[1]]
print(outpath)
ggsave(filename = outpath, plot = wc_vs_hudson, device = 'pdf', width = 8, height = 9, units = 'in', dpi = 300)

### Dependence of Fst on sample size

In [132]:
ggplot(fst_df_withSampleSize, aes(x = nmin, y = fst, color = fst_type)) + 
    geom_point(size = 2, alpha = 0.5) +
    geom_smooth(method = 'loess', se = FALSE) + 
    xlab('Minimum sample size') + ylab('Fst') +
    theme_classic()


#### My take

- WC Fst generally higher than Hudson, as expected based on Bhatia (2013)
- Highest Fst estimates occur for cities where sample sizes are lowest, suggesting these estimates may be biased upward
    - Downsampling Toronto data will help us resolve this
- That being said, Fst estimates are generally low for seemingly reliable estimates. Often Fst < 0.05

## Euclidean distance from PCA

- Estimate Euclidean distance between urban and rural centroids by city

In [142]:
euclidean <- function(x1, y1, x2, y2){
    
    dist <- sqrt((x1 - x2)^2 + (y1 - y2)^2)
    return(dist)
}

In [133]:
# Load data with habitat info
habitat_info <- suppressMessages(
    read_delim(
        '../../sequencing-prep/resources/low1_sampleSheet.txt', 
                           delim = '\t')) %>% 
    dplyr::select(continent, range, city, pop, individual, site, sample)

In [135]:
# Load covariance matrix from PCAngsd
cov_mat <- suppressMessages(
    read_delim(
        '../results/population_structure/pcangsd/highErrorRemoved_4fold_maf0.05_pcangsd.cov', 
                      col_names = FALSE, delim = ' ')) %>% 
      as.matrix()

# Combine continent and habitat data with sample order from ANGSD
samples <- suppressMessages(
    read_table(
        '../results/program_resources/angsd_highErrorRemoved_order.txt', col_names = FALSE) %>% 
  rename('sample' = 'X1')) %>%
  left_join(., habitat_info, by = 'sample')

In [136]:
# Dataframe with eigenvectors
eigenvectors <- eigen(cov_mat)
eigen_df <- eigenvectors$vectors %>% 
    as.data.frame() %>% 
    dplyr::select(V1, V2) %>% 
    rename('PC1' = 'V1',
         'PC2' = 'V2') %>% 
    bind_cols(., samples) %>% 
    mutate(sample_set = 'highErrorRemoved')

In [144]:
euc_dist_df <- eigen_df %>% 
    group_by(city, site) %>% 
    summarise(x = mean(PC1),
              y = mean(PC2)) %>% 
    pivot_wider(names_from = site, values_from = c(x, y)) %>% 
    mutate(distance = euclidean(x_u, y_u, x_r, y_r)) %>% 
    dplyr::select(city, distance)

In [145]:
head(euc_dist_df)

In [158]:
df_allStats <- euc_dist_df %>% 
    left_join(., fst_df %>% filter(fst_type == 'hudson'), by = 'city') %>% 
    left_join(., div_df, by = 'city') %>% 
    left_join(., slopes, by = 'city')

In [161]:
# Does Fst predict HCN?
fst_mod <- lm(betaHCN ~ fst, data = df_allStats)
summary(fst_mod)

In [162]:
# Does euclidean distance predict HCN?
dist_mod <- lm(betaHCN ~ distance, data = df_allStats)
summary(dist_mod)

## Important considerations

1. Some estimates of pairwise diversity Fst result from urban/rural populations with different sample sizes. It seems common to downsample populations to the lowest sample size before estimating Fst. I haven't seen a detailed examination of biases in Fst resulting from unequal sample sizes but it's common enough that I wonder if we should consider this. However, variation in sample sizes seems to be a bigger problem for WC Fst (Bhatia 2013) than Hudson's, so maybe we don't need to worry about it. Not really sure but something to think about. 
2. We still need to downsample the Toronto data to get a sense of whether our estimates of diversity and Fst are biased at lower sample sizes, and by how much. 
3. Estimates of differences in pi, Fst, and euclidean distance seem pretty small on average. If we wanted to get a sense of whether these estimates differ significantly from zero, I think a permutation approach would be most appropriate. We could generate 100 bootstrapped replicates of the one-dimensional (or 2D) SFS, re-calculated theta (or Fst), and generate confidence intervals around our estimates to see if they overlap zero. 